# Generating SLC stacks with topsStack

**Authors**: Bryan Riel

In this notebook, we will briefly demonstrate how to use `topsStack` to generate a stack of co-registered Sentinel-1 SLCs. The processing setup is identical to normal `topsApp.py` processing in that we download the appropriate SLC data, prepare a DEM for our study area, and download the necessary orbit and aux files from ASF/ESA (see previous tutorials on processing in TOPS mode with `topsApp.py`). More info on preparing for stacks can be found [here](https://github.com/parosen/Geo-SInC/tree/main/EarthScope2023/5.4_Intro_to_preparing_data_for_stack_processing).

<br>
<div class="alert alert-danger">
<font size="4"> <b> <font color='rgba(200,0,0,0.2)'> DISCLAIMER:  </font> </b> </font>

The following notebook has been run offline and is for demonstration purposes only. In order to run this notebook for your own data, you will need to download SLCs into an `asf` directory, place all orbits in an `orbits` directory, and all other auxiliary files in the `aux` directory. </i></b>
</div>

## Study area and setup

In this tutorial, our study area is Pine Island Glacier (PIG) in West Antartica. PIG is a fast-flowing ice stream and is responsible for about 25% of the mass loss from Antarctica for the past few decades. Here, we will use Sentinel-1 A/B IW SLCs for path 65, frame 906 (ascending) for two dates: 2020-01-12 and 2020-01-18. Moreover, we will only use one swath (Swath 2), which contains most of the fast-flowing regions of PIG (see image in main dense offsets notebook).

For the digital elevation model (DEM), we will use the Reference Elevation Model of Antarctica (REMA) 100 meter mosaic, which can be accessed here (https://www.pgc.umn.edu/data/rema/). We warp the REMA data from Polar Stereographic South to WGS84 and apply a geoid correction in order to use for ISCE processing. 

Let's first import necessary Python packages.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import datetime
import glob
import sys
import os
from osgeo import gdal, osr
import scipy.ndimage as ndimage
import isce

# Set environment variables for ISCE
isce_path = os.path.join(isce.__path__[0].strip(), 'applications')
os.environ['PATH'] = f"{isce_path}:{os.environ['PATH']}"

# setup path for topsStack
# $ISCE_STACK is set by conda
sys.path.insert(0, os.environ['ISCE_STACK'])
stack_path = os.path.join(os.environ['ISCE_STACK'], 'topsStack')
os.environ['PATH'] = f"{stack_path}:{os.environ['PATH']}"
os.environ['PYTHONPATH'] = f"{os.environ['ISCE_STACK']}:{os.environ['PYTHONPATH']}"

os.environ['OMP_NUM_THREADS'] = '8'
plt.rc('font', size=13)
parent_dir = os.getcwd()

Let's explore our top-level directory structure. Here, the S1A/B SLC zip files have been downloaded into the `asf` directory, and the orbit and auxiliary data have been downloaded into the `orbits` and `aux` directories, respectively.

In [7]:
!ls asf
print('')
!ls orbits | tail
print('')
!ls aux

download-all-2023-08-09_14-38-48.py
S1A_IW_SLC__1SSH_20200112T043556_20200112T043623_030762_038717_3E04.zip
S1B_IW_SLC__1SSH_20200118T043514_20200118T043541_019866_025927_DD28.zip

S1B_OPER_AUX_POEORB_OPOD_20210317T005427_V20200122T225942_20200124T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T011125_V20200123T225942_20200125T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T012810_V20200124T225942_20200126T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T014657_V20200125T225942_20200127T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T020358_V20200126T225942_20200128T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T022131_V20200127T225942_20200129T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T023933_V20200128T225942_20200130T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T025713_V20200129T225942_20200131T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T031408_V20200130T225942_20200201T005942.EOF
S1B_OPER_AUX_POEORB_OPOD_20210317T033059_V20200131T225942_20200202T005942.EOF

S1A_AUX_CAL_V20190228T092500_G20210104

### Running stackSentinel.py

We use `stackSentinel.py` to generate all configuration and run files required to be executed for a stack of Sentinel-1 TOPS data. Let's first make sure we can find it in our path:

In [2]:
!which stackSentinel.py

/home/jovyan/.local/envs/earthscope_insar_opencv/share/isce2/topsStack/stackSentinel.py


We now point `stackSentinel.py` to our input data and relevant processing options:
- `-W offset`: prepare files for generating dense offsets for various date pairs
- `-c 2`: generate a max of 2 offset pairs between each date and subsequent date
- `--swath_num '2'`: process swath 2
- `-b': specify the bounding box
- `-p hh`: specify HH polarization

In [3]:
%%bash

(stackSentinel.py \
    -s asf \
    -o orbits \
     -a aux \
    -d pig_rema_100m_filled_wgs84.dem \
    -W offset \
    -c 2 \
    --swath_num '2' \
    -b '-75.6 -74.4 -104 -96' \
    -p hh \
    --num_proc 4 \
    --num_proc4topo 1)

Number of SAFE files found: 2
Number of SAFE files to be used (cover BBOX): 2
*****************************************
Number of dates : 2
List of dates : 
['20200112', '20200118']
date      south      north
20200112 -76.739288 -73.961639
20200118 -76.737923 -73.960251
*****************************************
The overlap region among all dates (based on the preview kml files):
 South   North   East  West 
-76.737923 -73.961639 -105.927849 -95.770554
*****************************************
All dates (2)
['20200112', '20200118']

dates covering the bbox (2)
['20200112', '20200118']

The reference date was not chosen. The first date is considered as reference date.

All SLCs will be coregistered to : 20200112
secondary dates :
['20200118']

No existing stack was identified. A new stack will be generated.
selecting pairs with 2 nearest neighbor connections: 1
*****************************************
Coregistration method:  NESD
Workflow:  offset
***************************************

In [11]:
# Make sure run files are executable
!chmod a+x run_files/run_*

### Stack processing steps

We'll now go through each step of the processing by calling the run files one-by-one.

In [6]:
# Run unpack and 'topo' step: this maps the DEM into the reference image geometry at burst overlaps.
# This data is used for the co-registration of secondary images into the reference geometry
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_01_unpack_topo_reference', shell=True)

Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/testing/stack/asf/S1A_IW_SLC__1SSH_20200112T043556_20200112T043623_030762_038717_3E04.zip', '--swaths', '2', '--orbitdir', '/home/jovyan/testing/stack/orbits', '--outdir', '/home/jovyan/testing/stack/reference', '--auxdir', '/home/jovyan/testing/stack/aux', '--bbox', '-75.6 -74.4 -104 -96', '--pol', 'hh']
Input XML files:  ['/vsizip//home/jovyan/testing/stack/asf/S1A_IW_SLC__1SSH_20200112T043556_20200112T043623_030762_038717_3E04.zip/S1A_IW_SLC__1SSH_20200112T043556_20200112T043623_030762_038717_3E04.SAFE/annotation/s1a-iw2-slc-hh-20200112t043556-20200112t043621-030762-038717-002.xml']
Input TIFF files:  ['/vsizip//home/jovyan/testing/stack/asf/S1A_IW_SLC__1SSH_20200112T043556_20200112T043623_030762_038717_3E04.zip/S1A_IW_SLC__1SSH_20200112T043556_20200112T043623_030762_038717_3E04.SAFE/measurement/s1a-iw2-slc-hh-20200112t043556-20200112t04

CompletedProcess(args='./run_files/run_01_unpack_topo_reference', returncode=0)

In [6]:
# This unpacks all of the secondary SLC metadata
subprocess.run('./run_files/run_02_unpack_secondary_slc', shell=True)

CompletedProcess(args='./run_files/run_02_unpack_secondary_slc', returncode=0)

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: Sentinel1_TOPS
['--dirname', '/home/jovyan/testing/stack/asf/S1B_IW_SLC__1SSH_20200118T043514_20200118T043541_019866_025927_DD28.zip', '--swaths', '2', '--orbitdir', '/home/jovyan/testing/stack/orbits', '--outdir', '/home/jovyan/testing/stack/secondarys/20200118', '--auxdir', '/home/jovyan/testing/stack/aux', '--bbox', '-75.6 -74.4 -104 -96', '--pol', 'hh']
Input XML files:  ['/vsizip//home/jovyan/testing/stack/asf/S1B_IW_SLC__1SSH_20200118T043514_20200118T043541_019866_025927_DD28.zip/S1B_IW_SLC__1SSH_20200118T043514_20200118T043541_019866_025927_DD28.SAFE/annotation/s1b-iw2-slc-hh-20200118t043514-20200118t043539-019866-025927-002.xml']
Input TIFF files:  ['/vsizip//home/jovyan/testing/stack/asf/S1B_IW_SLC__1SSH_20200118T043514_20200118T043541_019866_025927_DD28.zip/S1B_IW_SLC__1SSH_20200118T043514_20200118T043541_019866_025927_DD28.SAFE/measurement/s1b-iw2-slc-hh-20200118t043514-20200118t043539

In [7]:
# Compute the average baseline between the secondary and reference orbits
subprocess.run('./run_files/run_03_average_baseline', shell=True)

CompletedProcess(args='./run_files/run_03_average_baseline', returncode=0)

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/home/jovyan/testing/stack/reference/', '--secondary', '/home/jovyan/testing/stack/secondarys/20200118', '--baseline_file', '/home/jovyan/testing/stack/baselines/20200112_20200118/20200112_20200118.txt']
minSecondary,maxSecondary 1 9
minReference,maxReference 1 9
minBurst, maxBurst:  1 9
Bprep:  [-173.00457565208308, -173.1278098246305, -173.25000218260953, -173.37003555542904, -173.48860040280883, -173.60559114498997, -173.72153906100343, -173.83528429056832, -173.94809339328572]
Bpar:  [100.16803877477841, 100.26919046739856, 100.36984654772266, 100.46942838608032, 100.56830439257017, 100.66637488829613, 100.76414837519742, 100.86062885914464, 100.95669805858446]
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Poly

In [8]:
# Extract SLC data over burst overlaps
subprocess.run('./run_files/run_04_extract_burst_overlaps', shell=True)

Overlap directory /home/jovyan/testing/stack/geom_reference/overlap/IW2 already exists
Subreference Overlap directory /home/jovyan/testing/stack/reference/overlap/IW2 already exists
Overlap 0 start time - IW-2 2020-01-12 04:35:59.462673 subset
Overlap 0 stop time - IW-2 2020-01-12 04:35:59.799784 subset
Overlap 0 number of lines - IW-2 165 subset
Overlap 0 number of valid lines - IW-2 120 subset
Top:  ['/home/jovyan/testing/stack/reference/overlap/IW2/burst_top_01_02.slc']
Bottom:  ['/home/jovyan/testing/stack/reference/overlap/IW2/burst_bot_01_02.slc']
Overlap 1 start time - IW-2 2020-01-12 04:36:02.225341 subset
Overlap 1 stop time - IW-2 2020-01-12 04:36:02.556285 subset
Overlap 1 number of lines - IW-2 162 subset
Overlap 1 number of valid lines - IW-2 117 subset
Top:  ['/home/jovyan/testing/stack/reference/overlap/IW2/burst_top_01_02.slc', '/home/jovyan/testing/stack/reference/overlap/IW2/burst_top_02_03.slc']
Bottom:  ['/home/jovyan/testing/stack/reference/overlap/IW2/burst_bot_01

CompletedProcess(args='./run_files/run_04_extract_burst_overlaps', returncode=0)

In [9]:
# Compute mapping from DEM (geographic) coordinates to radar burst coordinates for each secondary SLC
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_05_overlap_geo2rdr', shell=True)

CompletedProcess(args='./run_files/run_05_overlap_geo2rdr', returncode=0)

API open (WR): /home/jovyan/testing/stack/coreg_secondarys/20200118/overlap/IW2/range_top_01_02.off
API open (WR): /home/jovyan/testing/stack/coreg_secondarys/20200118/overlap/IW2/azimuth_top_01_02.off
GDAL open (R): /home/jovyan/testing/stack/geom_reference/overlap/IW2/hgt_01_02.rdr.vrt
GDAL open (R): /home/jovyan/testing/stack/geom_reference/overlap/IW2/lat_01_02.rdr.vrt
GDAL open (R): /home/jovyan/testing/stack/geom_reference/overlap/IW2/lon_01_02.rdr.vrt
 Orbit interpolation method: hermite
 threads           8
 Starting Acquisition time:    16514.606056000001     
 Stop Acquisition time:    16517.699668231500     
 Azimuth line spacing in secs:    2.0555562999999980E-003
 Near Range in m:    861300.14253605576     
 Far  Range in m:    916543.37852441496     
 Range sample spacing in m:    2.3295621147153232     
 Radar Image Lines:         1506
 Radar Image Width:        23715
 reading dem ...
 Geocoded Lines:           120
 Geocoded Samples:       23715
 Dopplers:    0.000000000

In [10]:
# Resample secondary bursts to reference geometry
subprocess.run('./run_files/run_06_overlap_resample', shell=True)

CompletedProcess(args='./run_files/run_06_overlap_resample', returncode=0)

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/testing/stack/secondarys/20200118', '--reference', '/home/jovyan/testing/stack/reference', '--coregdir', '/home/jovyan/testing/stack/coreg_secondarys/20200118', '--overlap']
Estimated burst offset:  0
Shifts:  {0: 0, 1: 0, 2: 0, 3: -1, 4: 0, 5: 1, 6: -4, 7: 0, 8: -3}
Chi squared: 0.000345
Misfit radians - Max: 0.0012790650216629729 , Min : -0.0006733829541190062 
True
2023-08-14 16:44:40,232 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1506 which is the number of lines in the slc image.
2023-08-14 16:44:40,232 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_WIDTH has been set to the default value 23715 which is the width of the slc image.
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 5 - by - 3 
11133.7	-395.498	-35.4165	-0.257415	
-44357.2	1734.49	36

In [11]:
# Compute residual offsets between reference and secondary bursts over the overlaps
subprocess.run('./run_files/run_07_pairs_misreg', shell=True)

CompletedProcess(args='./run_files/run_07_pairs_misreg', returncode=0)

GDAL open (R): /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.int.vrt
GDAL close: /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.int.vrt
GDAL open (R): /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.int.vrt
API open (WR): /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.5alks_15rlks.int
GDAL close: /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.int.vrt
API close:  /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.5alks_15rlks.int
API open (R): /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.5alks_15rlks.int
API close:  /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.5alks_15rlks.int
Output filename : /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.5alks_15rlks.int
Writing geotrans to VRT for /home/jovyan/testing/stack/ESD/20200112_20200118/IW2/overlap_01.5alks_15rlks.int
Completed Parsing the Configuration file
Functions to be executed:
['Func

In [3]:
# Compute time series of offsets for temporal consistency
subprocess.run('./run_files/run_08_timeseries_misreg', shell=True)


Rank of design matrix: 1
Design matrix is full rank.
RMSE : 2.704413776255521e-11 pixels

Estimated offsets with respect to the stack reference date

20200112 : 0.0
20200118 : 0.0009074507085964287


Rank of design matrix: 1
Design matrix is full rank.
RMSE : 2.9289246139541092e-08 pixels

Estimated offsets with respect to the stack reference date

20200112 : 0.0
20200118 : 0.9827840464347731



CompletedProcess(args='./run_files/run_08_timeseries_misreg', returncode=0)

In [5]:
# Compute mapping from DEM to radar coordinates for the full bursts for the secondary SLCs
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_09_fullBurst_geo2rdr', shell=True)

CompletedProcess(args='./run_files/run_09_fullBurst_geo2rdr', returncode=0)

API open (WR): /home/jovyan/testing/stack/coreg_secondarys/20200118/IW2/range_01.off
API open (WR): /home/jovyan/testing/stack/coreg_secondarys/20200118/IW2/azimuth_01.off
GDAL open (R): /home/jovyan/testing/stack/geom_reference/IW2/hgt_01.rdr.vrt
GDAL open (R): /home/jovyan/testing/stack/geom_reference/IW2/lat_01.rdr.vrt
GDAL open (R): /home/jovyan/testing/stack/geom_reference/IW2/lon_01.rdr.vrt
 Orbit interpolation method: hermite
 threads           8
 Starting Acquisition time:    16514.606054000000     
 Stop Acquisition time:    16517.699666231500     
 Azimuth line spacing in secs:    2.0555562999999980E-003
 Near Range in m:    861299.15975200932     
 Far  Range in m:    916542.39574036852     
 Range sample spacing in m:    2.3295621147153232     
 Radar Image Lines:         1506
 Radar Image Width:        23715
 reading dem ...
 Geocoded Lines:          1506
 Geocoded Samples:       23715
 Dopplers:    0.0000000000000000        0.0000000000000000     
 geo2rdr on            8

In [6]:
# Resample secondary SLC bursts to reference
os.environ['OMP_NUM_THREADS'] = '8'
subprocess.run('./run_files/run_10_fullBurst_resample', shell=True)

CompletedProcess(args='./run_files/run_10_fullBurst_resample', returncode=0)

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/home/jovyan/testing/stack/secondarys/20200118', '--reference', '/home/jovyan/testing/stack/reference', '--coregdir', '/home/jovyan/testing/stack/coreg_secondarys/20200118', '--azimuth_misreg', '/home/jovyan/testing/stack/misreg/azimuth/dates/20200118.txt', '--range_misreg', '/home/jovyan/testing/stack/misreg/range/dates/20200118.txt']
Estimated burst offset:  0
Shifts:  {0: 0, 1: 0, 2: 0, 3: -1, 4: 0, 5: 1, 6: -4, 7: 0, 8: -3}
Chi squared: 0.000345
Misfit radians - Max: 0.0012790650216629729 , Min : -0.0006733829541190062 
True
2023-08-14 17:47:04,250 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_LINES has been set to the default value 1506 which is the number of lines in the slc image.
2023-08-14 17:47:04,250 - isce.stdproc.resamp_slc - WARNING - The variable INPUT_WIDTH has been set to the default value 23715 which is the width of the slc image.
P

In [7]:
# Check the valid bursts (bursts that exist over all images)
subprocess.run('./run_files/run_11_extract_stack_valid_region', shell=True)

creating  /home/jovyan/testing/stack/stack
checking the number of bursts in coreg_secondarys against the one in reference
all secondary images have the same number of bursts as the reference
******************
swath:  2
/home/jovyan/testing/stack/coreg_secondarys/20200118
minSecondary,maxSecondary 1 9
minReference,maxReference 1 9
minBurst, maxBurst:  1 9
writing  /home/jovyan/testing/stack/stack/IW2.xml


CompletedProcess(args='./run_files/run_11_extract_stack_valid_region', returncode=0)

In [8]:
# Compute fully merged SLC images
# This is the last step before dense offsets
subprocess.run('./run_files/run_12_merge_reference_secondary_slc', shell=True)

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/testing/stack/stack', '--inp_reference', '/home/jovyan/testing/stack/reference', '--dirname', '/home/jovyan/testing/stack/reference', '--name_pattern', 'burst*slc', '--outfile', '/home/jovyan/testing/stack/merged/SLC/20200112/20200112.slc', '--method', 'top', '--valid_only', '--range_looks', '9', '--azimuth_looks', '3']
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 9
writing merged file to disk via gdal.Translate ...
00...10...21..30...41...50...60...71...80...91..100 
Skipping multi-looking ....
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/home/jovyan/testing/stack/stack', '--inp_reference', '/home/jovyan/testing/stack/reference', '--dirname', '/home/jovyan/testing/stack/geom_reference', '--name_pattern', 'lat*rdr', '--outfile', '/home/jovyan/

CompletedProcess(args='./run_files/run_12_merge_reference_secondary_slc', returncode=0)

In [9]:
!ls -l merged/SLC/*

merged/SLC/20200112:
total 2267960
-rw-r--r-- 1 jovyan users 2322362520 Aug 14 17:53 20200112.slc.full
-rw-r--r-- 1 jovyan users        537 Aug 14 17:53 20200112.slc.full.aux.xml
-rw-r--r-- 1 jovyan users       3986 Aug 14 17:51 20200112.slc.full.vrt
-rw-r--r-- 1 jovyan users       4125 Aug 14 17:51 20200112.slc.full.xml
-rw-r--r-- 1 jovyan users        266 Aug 14 17:53 20200112.slc.hdr

merged/SLC/20200118:
total 2267964
-rw-r--r-- 1 jovyan users 2322362520 Aug 14 17:55 20200118.slc.full
-rw-r--r-- 1 jovyan users        537 Aug 14 17:55 20200118.slc.full.aux.xml
-rw-r--r-- 1 jovyan users       4130 Aug 14 17:55 20200118.slc.full.vrt
-rw-r--r-- 1 jovyan users       4329 Aug 14 17:55 20200118.slc.full.xml
-rw-r--r-- 1 jovyan users        266 Aug 14 17:55 20200118.slc.hdr


In [10]:
!ls -l merged/geom_reference

total 12053832
-rw-r--r-- 1 jovyan users        256 Aug 14 17:54 hgt.hdr
-rw-r--r-- 1 jovyan users   86006400 Aug 14 17:54 hgt.rdr
-rw-r--r-- 1 jovyan users        535 Aug 14 17:54 hgt.rdr.aux.xml
-rw-r--r-- 1 jovyan users 2322362520 Aug 14 17:54 hgt.rdr.full
-rw-r--r-- 1 jovyan users        537 Aug 14 17:54 hgt.rdr.full.aux.xml
-rw-r--r-- 1 jovyan users       3930 Aug 14 17:54 hgt.rdr.full.vrt
-rw-r--r-- 1 jovyan users       4186 Aug 14 17:54 hgt.rdr.full.xml
-rw-r--r-- 1 jovyan users        707 Aug 14 17:54 hgt.rdr.vrt
-rw-r--r-- 1 jovyan users        269 Aug 14 17:55 incLocal.hdr
-rw-r--r-- 1 jovyan users   86006400 Aug 14 17:55 incLocal.rdr
-rw-r--r-- 1 jovyan users        633 Aug 14 17:55 incLocal.rdr.aux.xml
-rw-r--r-- 1 jovyan users 2322362520 Aug 14 17:55 incLocal.rdr.full
-rw-r--r-- 1 jovyan users        635 Aug 14 17:55 incLocal.rdr.full.aux.xml
-rw-r--r-- 1 jovyan users       7885 Aug 14 17:55 incLocal.rdr.full.vrt
-rw-r--r-- 1 jovyan users       4366 Aug 14 17:55 incLocal.r

We now have all the files necessary to generate dense offsets. Unlike `topsApp.py`, the `run_13_dense_offsets` step only generates the offsets in radar coordinates and does not perform any filtering or geocoding. Please see the main dense offsets notebook for a complete processing workflow.